In [ ]:
import random
import time

import diffusers
import torch

# To test performance delta.
use_cuda = torch.cuda.is_available()

def inference(prompt, n_images, guidance, steps, width, height, seed, neg_prompt):
    generator = None
    if use_cuda:
        generator = torch.Generator("cuda").manual_seed(seed) if seed else None
    elif seed:      
        generator = torch.Generator()
        generator.manual_seed(seed)
    # There's no fp32 revision.
    # https://huggingface.co/stabilityai/stable-diffusion-2-1/tree/main
    # https://huggingface.co/stabilityai/stable-diffusion-2-1/resolve/fp16/model_index.json
    pipe = diffusers.StableDiffusionPipeline.from_pretrained(
        "stabilityai/stable-diffusion-2-1",
        revision="fp16",
        torch_dtype=torch.float16 if use_cuda else torch.float32)
    pipe.scheduler = diffusers.DPMSolverMultistepScheduler.from_pretrained(
        "stabilityai/stable-diffusion-2-1", subfolder="scheduler")
    if use_cuda:
        pipe = pipe.to("cuda")
        pipe.enable_attention_slicing()
    return pipe(
        prompt,
        num_images_per_prompt=n_images,
        negative_prompt=neg_prompt,
        num_inference_steps=steps,
        guidance_scale=guidance,
        width=width,
        height=height,
        generator=generator).images

def run(prompt):
    n_images = 1 # number of images
    neg_prompt = "Disfigured. Too many fingers."
    guidance = 7.5 # max = 15
    steps = 25 # [2, 100]
    width = 768  # [64, 1024] step=8
    height = 768 # [64, 1024] step=8
    seed = 12
    #seed = random.randint(0, 2147483647)

    out = "%s_steps%d_guidance%.1f_seed%d.png" % (prompt.replace(".", ""), steps, guidance, seed)
    print(f"Seed: {seed}")
    start = time.time()
    gallery = inference(prompt, n_images, guidance, steps, width, height, seed, neg_prompt)
    print("Took %.1fs" % (time.time()-start))
    gallery[0].save("out/"+out)
    return gallery[0]

prompt = "a squirrel cat hybrid"
prompt = "Old man making clockwork children toys in his workshop. Happy place. Holidays."
img = run(prompt)
img